In [1]:
import pandas as pd 
import numpy as np 

In [2]:
datos = pd.read_excel("datos/Base_Datos_Presidencia_Cam (4).xlsx",skiprows=12,sheet_name=0)

datos.TIPO_VALORACION=datos.TIPO_VALORACION.map({"SIN VALORACIÓN":"S","POSITIVA":"P","NEGATIVA":"N"})

In [3]:
def tiempoSegundos(x):
    tiempo_separado = x.split(":")
    segundos = 3600 * int(tiempo_separado[0]) + 60 * int(tiempo_separado[1]) + int(tiempo_separado[2])
    return segundos

In [4]:

datos["DURACION_SEGUNDOS"] = datos.DURACION_PIEZA.apply(lambda x : tiempoSegundos(x))

datos["DURACION_MINUTOS"] = datos["DURACION_SEGUNDOS"]/60
datos["Fecha"]=pd.to_datetime(datos.FECHA_TESTIGO, format="%d/%m/%Y")
datos.sort_values(by="Fecha").FECHA_TESTIGO.unique()

array(['01/03/2024', '02/03/2024', '03/03/2024', '04/03/2024',
       '05/03/2024', '06/03/2024', '07/03/2024', '08/03/2024',
       '09/03/2024', '10/03/2024', '11/03/2024', '12/03/2024',
       '13/03/2024', '14/03/2024', '15/03/2024', '16/03/2024',
       '17/03/2024', '18/03/2024', '19/03/2024', '20/03/2024',
       '21/03/2024', '22/03/2024', '23/03/2024', '24/03/2024',
       '25/03/2024', '26/03/2024', '27/03/2024', '28/03/2024',
       '29/03/2024', '30/03/2024', '31/03/2024', '01/04/2024',
       '02/04/2024', '03/04/2024', '04/04/2024', '05/04/2024',
       '06/04/2024', '07/04/2024', '08/04/2024', '09/04/2024',
       '10/04/2024', '11/04/2024', '12/04/2024', '13/04/2024',
       '14/04/2024', '15/04/2024', '16/04/2024', '17/04/2024',
       '18/04/2024', '19/04/2024', '20/04/2024', '21/04/2024',
       '22/04/2024', '23/04/2024', '24/04/2024', '25/04/2024',
       '26/04/2024', '27/04/2024', '28/04/2024', '29/04/2024',
       '30/04/2024', '01/05/2024', '02/05/2024', '03/05

In [5]:
datos["FRASE_ESTEREOTIPO_GENERAL"]=datos[["PRESENCIA DE ESTEREOTIPOS","FRASE_ESTEREOTIPO_VULNERABILIDAD"]].apply(lambda x : x.dropna().unique()[0] if len(x.dropna().unique()) else np.nan,axis=1)

In [6]:
datos["FRASE_POSITIVA"] = datos.apply(lambda x :  x.ADJETIVO_FRASE if str(x.TIPO_VALORACION)=="P" else np.nan,axis=1)

datos["FRASE_NEGATIVA"] = datos.apply(lambda x :  x.ADJETIVO_FRASE if str(x.TIPO_VALORACION)=="N" else np.nan,axis=1)

In [7]:
def concatenar_frases(frases):
    frases_no_vacias = list(set(frase for frase in frases if frase.strip()))
    return '... <br/>'.join(frases_no_vacias) if frases_no_vacias else np.nan

In [8]:
def concatenar_frases_2(frases):

    frases_no_vacias = list(set(frase for frase in frases.fillna("") if frase.strip()))
    return '... <br/>'.join(frases_no_vacias) if frases_no_vacias else np.nan

In [9]:
datum = datos.sort_values(by="Fecha",ascending=False)[["NOMBRE_NOTICIERO","ID_PIEZA_INFORMATIVA","FECHA_TESTIGO","TIPO_VALORACION","CANDIDATO","DURACION_MINUTOS","FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]]

datos_frases = datum.loc[datum[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].dropna(how="all").index]
datos_sin_frases = datum.loc[[i for i in  datum.index if i not in datos_frases.index]]

datos_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]] = datos_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].fillna("")
datos_sin_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]] = datos_sin_frases[["FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]].fillna("")

datum_frases_agrupadas = datos_frases.groupby(["NOMBRE_NOTICIERO",'ID_PIEZA_INFORMATIVA', 'FECHA_TESTIGO',
       'TIPO_VALORACION', 'CANDIDATO', 'DURACION_MINUTOS']).agg(concatenar_frases).reset_index()
datum_frases_agrupadas = datum_frases_agrupadas.groupby(["NOMBRE_NOTICIERO",'ID_PIEZA_INFORMATIVA', 'FECHA_TESTIGO','TIPO_VALORACION', 'CANDIDATO'])
datum_frases_agrupadas = datum_frases_agrupadas.agg({'FRASE_ESTEREOTIPO_GENERAL':concatenar_frases_2, 'FRASE_POSITIVA':concatenar_frases_2, 'FRASE_NEGATIVA':concatenar_frases_2,'FRASE_ENFOQUE':concatenar_frases_2,'DURACION_MINUTOS':sum},axis=1).reset_index()




datum_sin_frases_agrupadas=datos_sin_frases.groupby(["NOMBRE_NOTICIERO","ID_PIEZA_INFORMATIVA","FECHA_TESTIGO","TIPO_VALORACION","CANDIDATO","FRASE_ESTEREOTIPO_GENERAL","FRASE_POSITIVA","FRASE_NEGATIVA","FRASE_ENFOQUE"]).sum().reset_index()

ladata = pd.concat([datum_frases_agrupadas,datum_sin_frases_agrupadas])

dict_not_id = {}
dict_id_not ={}
a=1
for noticiero in ladata.NOMBRE_NOTICIERO.unique():
    dict_not_id[noticiero] = a
    dict_id_not[a] = noticiero
    a+=1
ladata["ID_NOTICIERO"] = ladata.NOMBRE_NOTICIERO.map(dict_not_id)

ladata.to_csv("datos_limpios/lista_medios.csv",index=False)
pd.DataFrame([dict_id_not]).T[0].to_json("datos_limpios/diccionario_medios.json")


In [10]:
datum_frases_agrupadas.shape

(3782, 10)

In [11]:
datum_frases_agrupadas.columns

Index(['NOMBRE_NOTICIERO', 'ID_PIEZA_INFORMATIVA', 'FECHA_TESTIGO',
       'TIPO_VALORACION', 'CANDIDATO', 'FRASE_ESTEREOTIPO_GENERAL',
       'FRASE_POSITIVA', 'FRASE_NEGATIVA', 'FRASE_ENFOQUE',
       'DURACION_MINUTOS'],
      dtype='object')

In [12]:
ef = datum_frases_agrupadas.groupby(["NOMBRE_NOTICIERO",'ID_PIEZA_INFORMATIVA', 'FECHA_TESTIGO','TIPO_VALORACION', 'CANDIDATO'])
ef = ef.agg({'FRASE_ESTEREOTIPO_GENERAL':concatenar_frases_2, 'FRASE_POSITIVA':concatenar_frases_2, 'FRASE_NEGATIVA':concatenar_frases_2,'FRASE_ENFOQUE':concatenar_frases_2,'DURACION_MINUTOS':sum},axis=0).reset_index()


ef[ef.FRASE_POSITIVA.notna()]

,NOMBRE_NOTICIERO,ID_PIEZA_INFORMATIVA,FECHA_TESTIGO,TIPO_VALORACION,CANDIDATO,FRASE_ESTEREOTIPO_GENERAL,FRASE_POSITIVA,FRASE_NEGATIVA,FRASE_ENFOQUE,DURACION_MINUTOS
6,1 NOTICIAS,63007,16/05/2024,P,CSP,NaN,"Y le mencionábamos nosotros que, con todo este...",NaN,NaN,0.650000
9,A LA 1,7417,01/03/2024,P,BXGR,NaN,"Xóchitl Gálvez arrancó, arrancó fuerte en Fres...",NaN,NaN,0.616667
12,A LA 1,8087,01/03/2024,P,CSP,NaN,Claudia Sheinbaum subió su primer spot en el q...,NaN,NaN,0.450000
13,A LA 1,14834,11/03/2024,P,BXGR,NaN,"Por cierto, la señora Xóchitl Gálvez subiendo ...",NaN,"Por cierto, la señora Xóchitl Gálvez subiendo ...",0.283333
20,A LA 1,42567,17/04/2024,P,CSP,NaN,Claudia Sheinbaum llegó al Estado de Yucatán d...,NaN,NaN,1.200000
...,...,...,...,...,...,...,...,...,...,...
3759,ZONA 3 TERCERA EMISIÓN,40333,15/04/2024,P,JAM,NaN,"O sea, hasta ahí se puede notar cómo llega la ...",NaN,NaN,1.366667
3766,ZONA 3 TERCERA EMISIÓN,65587,20/05/2024,P,CSP,NaN,"Es cierto, es ella la, la, la única que ha est...",NaN,"Es cierto, es ella la, la, la única que ha est...",0.266667
3767,ZONA DE NOTICIAS,49186,28/04/2024,P,CSP,NaN,Y es que los temas que se van a abordar son ya...,NaN,Y es que los temas que se van a abordar son ya...,1.333333
3769,ZONA DE NOTICIAS,58883,11/05/2024,P,CSP,NaN,"Dijo que ahora, a partir de 2025, en el caso d...",NaN,"Dijo que ahora, a partir de 2025, en el caso d...",0.900000
